In [1]:
import numpy as np
from scipy import linalg as la

In [2]:
points = np.random.uniform(-1, 1, (2, 2000))

In [3]:
points

array([[ 0.4324664 , -0.71518971,  0.77093954, ...,  0.35551768,
        -0.99855736, -0.39120673],
       [ 0.38063672, -0.86454305, -0.71433273, ..., -0.60312056,
         0.8092171 ,  0.52420147]])

In [4]:
lengths = la.norm(points, axis=0)

In [5]:
num_within = np.count_nonzero(lengths < 1)

In [6]:
4 * (num_within/2000)

3.154

### Calculating $\pi$ with Monte Carlo simulation

In [7]:
def multi_pi(n, *, N = 10_000):
    
    """ 
    This function estimates a multidimensional pi values given a dimension n and a drawing sample N
    """
    
    import numpy as np
    from scipy import linalg as la
    
    points = np.random.uniform(-1, 1, (n, N))
    
    lengths = la.norm(points, axis=0)
    num_within = np.count_nonzero(lengths < 1)
    
    # The volume of [-1, 1]x[-1, 1]x... = 2^n
    
    return np.power(2, n) * (num_within/N)

In [8]:
multi_pi(2)

3.1296

In [9]:
multi_pi(3)

4.2896

In [10]:
multi_pi(4)

4.9552

### One dimensional Monte Carlo integration function

The idea is that:

$$ \int_{a}^{b} f(x) dx = (b-a) f_{avg} \approx V(\Omega) \frac{1}{N} \sum_{i=1}^{N} f(x_i) $$

In [11]:
def one_var_inetgration(f, a:float, b:float, *, N = 10_000):
    
    """
    This function integrates over a given interval [a, b] using a given function f
    """
    
    import numpy as np
    from scipy import linalg as la
    
    if a > b:
        a, b = b, a
    else:
        pass
    
    #The 'volume':
    V = b - a
    
    #Generating random numbers on the interval N times:
    points = np.random.uniform(a, b, (1, N))
    
    #Summing over the randomly generated function outputs:
    s = np.sum(f(points))
    summa = s/N
    
    return V * summa

In [12]:
f = lambda x: x**2
one_var_inetgration(f, -4, 2)

24.119760027425812

In [13]:
f = lambda x: np.sin(x)
one_var_inetgration(f, -2*np.pi, 2*np.pi, N = 1_000_000)

0.006154191923415477

In [14]:
f = lambda x: 1/x
one_var_inetgration(f, 1, 10, N = 1_000_000)

2.3030734792743015

In [15]:
f = lambda x: np.abs(np.sin(10*x)*np.cos(10*x) + np.sqrt(x)*np.sin(3*x))
one_var_inetgration(f, 1, 5, N = 1_000_000)

4.499377443160346

### Multi dimensional Monte Carlo integration function

In [16]:
def multi_var_inetgration(f, a:list, b:list, *, N = 10_000):
    
    """
    This function integrates over given intervals [a, b] using a given function f
    """
    
    import numpy as np
    from scipy import linalg as la
    
    a, b = np.array(a), np.array(b)
    
    #The 'volume':
    diff = b - a
    V = np.prod(b - a)
    
    #Generating random numbers on n dimensions, on the given intervals N times:
    n = len(a)
    points = np.random.uniform(0, 1, (n, N))
    
    for j in range(n-1):
        points[j] = points[j] * (b[j] - a[j]) + a[j]
    
    #Summing over the randomly generated function outputs:
    
    s = np.sum([f(*points[:, k]) for k in range(N)])
    
    summa = s/N
    
    return V * summa

In [17]:
f = lambda x, y: x**2 + y**2
multi_var_inetgration(f, [0, 0], [1, 1])

0.6658375659659472

In [18]:
f = lambda x, y: 3*x - 4*y + y**2
multi_var_inetgration(f, [1, -2], [3, 1], N = 1_000_000)

25.98009450911292

In [19]:
f = lambda x, y, w, z: x + y - w * z**2
multi_var_inetgration(f, [-1, -2, -3, -4], [1, 2, 3, 4], N = 1_000_000)

-0.7279777287480039

In [20]:
f = lambda x, y, z, w: 1/((2*np.pi)**2) * np.power(np.e, np.matmul(np.array([x, y, z, w]).T, np.array([x, y, z, w])))
multi_var_inetgration(f, [-3/2, 0, 0, 0], [3/4, 1, 1/2, 1], N = 1_000_000)

0.14728390869590055

In [21]:
from scipy import stats
# Define the bounds of integration.

mins = np.array([-3/2, 0, 0, 0])
maxs = np.array([3/4, 1, 1/2, 1])

# The distribution has mean 0 and covariance I (the n x n identity).
means, cov = np.zeros(4), np.eye(4)

# Compute the integral with SciPy.
stats.mvn.mvnun(mins, maxs, means, cov)[0]

0.015762404844861223